In [2]:
#1
#Regular 3 phase SFDI 
#OpenSFDI data
#calibrating system with reference phantoms given (blank images)
#getting absorption and scattering of three phase data after calibrating with blank images
import numpy as np
import matplotlib.pyplot as plt
import cv2
import matplotlib
from scipy.interpolate import griddata
from scipy.signal import hilbert

%matplotlib qt

#these parameters are given with the open source data
w = 0.00067 #mm (670nm)
sf = [0, 0.2] #spatial frequencies - mm^{-1}
p = [0, (2/3)*np.pi, (4/3)*np.pi]#for 3 phases[0, 120, 240]degrees


images = [] #raw data


def image_tif(sf, p): #read in raw data
    im1 = cv2.imread('im_1_' + str(sf+1) + '_' + str(p+1) + '.tif', 0)
    return im1.astype(np.double)

#putting the open source raw data into 'images'
for i in range(len(sf)):
    for j in range(len(p)):        
        image_tif(i, j)
        images.append(image_tif(i, j))
        
        
#Demodulation
def AC(var):
    return (2**(1/2)/3)*(((var[3]-var[4])**2 + (var[4]-var[5])**2 + (var[5]-var[3])**2)**(1/2))

def DC(var):
    return np.asarray((1/3)*(var[0]+var[1]+var[2]))

M_AC = AC(images) #AC modulation amplitude

M_DC = DC(images) #DC modulation amplitude



#we need to figure out diffuse reflectance of open source phantom images given the optical properties
images2 = []
ran = [1, 2, 3, 4, 5, 6]

#putting the open source raw data phantom images into 'images2'
for i in range(len(ran)):
    image_tif2 = cv2.imread('im0{}.tif'.format(i+1))
    images2.append(image_tif2.astype(np.double))

    

M_AC_ref = AC(images2) #AC modulation amplitude for reference measurement
M_DC_ref = DC(images2) #DC modulation amplitude for reference measurement


#taking the information wanted and putting the reference modulation amplitudes to 'M_AC_ref2' & 'M_DC_ref2'
M_AC_ref2 = []
M_DC_ref2 = []
for i in range(M_AC_ref.shape[0]):
    for j in range(M_AC_ref.shape[1]):
        M_AC_ref2.append(M_AC_ref[i][j][0])
        M_DC_ref2.append(M_DC_ref[i][j][0])


#reshaping the reference AC and DC modulation amplitudes so the image can be shown
M_AC_ref3 = np.reshape(M_AC_ref2, (M_AC.shape[0], M_AC.shape[1]))
M_DC_ref3 = np.reshape(M_DC_ref2, (M_DC.shape[0], M_DC.shape[1]))


#need R_d,ref,pred
#all these parameters are given in the open source data
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 


#calculating the reference diffusse reflectance AC value
f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

R_AC_ref = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

        
          
#calculating the diffuse reflectance DC value
f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

R_DC_ref = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))


#Diffuse reflectance = (modulation amplitude/reference modulation amplitude)*reference diffuse reflectance
R_d_AC = (M_AC/M_AC_ref3)*R_AC_ref
R_d_DC = (M_DC/M_DC_ref3)*R_DC_ref


#plotting the AC and DC diffuse reflectance values
plt.subplot(221), plt.imshow(R_d_AC), plt.colorbar(), plt.title('AC'), plt.xticks([]),  plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC), plt.colorbar(), plt.title('DC'), plt.xticks([]),  plt.yticks([])
     
    
#determining the optical properties from diffuse reflectance values
xi = []
#puts the DC and AC diffuse reflectance values into an array
for i in range(R_d_AC.shape[0]): 
    for j in range(R_d_AC.shape[1]):
        freq = [R_d_DC[i][j], R_d_AC[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001) #we are setting the absorption coefficient range
mu_sp = np.arange(0.3, 3, 0.01) #we are setting the reduces scattering coefficient range
n = 1.4 #refractive index of tissue 

#THE DIFFUSION APPROXIMATION
#getting the diffuse reflectance AC values corresponding to specific absorption and reduced scattering coefficients
Reflectance_AC = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
    
#getting the diffuse reflectance DC values corresponding to specific absorption and reduced scattering coefficients
#getting absorption and reduced scattering coefficients correponding the AC and DC diffuse reflectance values
Reflectance_DC = []
op_mua = []
op_sp = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


#putting the DC and AC diffuse reflectance values generated from the Diffusion Approximation into an array    
points = []
for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

#putting the optical properties into two seperate arrays
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array

#using scipy.interpolate.griddata to perform cubic interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_abs = griddata(points_array, values1, xi, method='cubic') #mua
coeff_sct = griddata(points_array, values2, xi, method='cubic') #musp

abs_plot = np.reshape(coeff_abs, (R_d_AC.shape[0], R_d_AC.shape[1]))
sct_plot = np.reshape(coeff_sct, (R_d_AC.shape[0], R_d_AC.shape[1]))


#plotting optical property maps
plt.subplot(223), plt.imshow(abs_plot), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([])
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot), plt.title('Reduced scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


(512, 672, 3)


D:\Users\janeh\anaconda\lib\site-packages\ipykernel_launcher.py:99: RuntimeWarning: divide by zero encountered in true_divide
D:\Users\janeh\anaconda\lib\site-packages\ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in true_divide


In [3]:
#2
#Filter & shift method
#SSOP to get optical properties
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('im_1_2_1.tif',0)).astype(np.double) #singular image

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

#2D Fourier Transform
f1 = np.fft.fft2(im1)
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1)) #this will be used for plotting purposes ONLY
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2)) #this will be used for plotting purposes ONLY


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)


#FOR IMAGE
#HIGH PASS FILTER
mask1 = FT1 
mask1[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking out the low frequency & LHS high frequency
mask1[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0 #mirror mask on other side of RHS high frequency
roll1 = np.roll(mask1, -diff1) #rolling the image so the high frequency point is at the centre of the image
IFT = np.abs(np.fft.ifft2(np.fft.ifftshift(roll1)))#inverse FFT


#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0 #masking out LHS high frequency
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0 #masking out RHS high frequency
IFT2 = np.abs((np.fft.ifft2(np.fft.ifftshift(maskl2))))



#we need to figure out diffuse reflectance of phantom given the optical properties
ref_im = (cv2.imread('im04.tif',0)).astype(np.double) #single image of phantom
f_ref = np.fft.fft2(ref_im) #2D DFT
#for high pass filter
FTh = np.fft.fftshift(f_ref)
#for low pass filter
FTl = np.fft.fftshift(f_ref)


#FOR IMAGE
#HIGH PASS FILTER
maskc = FTh 
maskc[0:height, lbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking out low frequency and LHS high frequency
maskc[0:height, rbp1+int(diff1/2):rbp1+int(diff1/2)+int(2*diff1)] = 0 #mirror mask on other side of RHS high frequency
rollc = np.roll(maskc, -diff1) #rolling image so RHS high frequency is in the centre
IFT_h = np.abs(np.fft.ifft2(np.fft.ifftshift(rollc))) #inverse FFT


#LOW PASS FILTER
maskd = FTl
maskd[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0 #masking LHS high frequency
maskd[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0 #masking RHS high frequency
IFT2_l = np.abs((np.fft.ifft2(np.fft.ifftshift(maskd))))



#Open source data given information
#Need to calculate reference diffuse reflectance and reference modulation amplitude values
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 

#calculating reference diffuse reflectance AC value
f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)
R_AC_ref_s = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

#calculating reference diffuse reflectance DC value       
f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)
R_DC_ref_s = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))

#Diffuse reflectance = (modulation amplitude/reference modulation amplitude)*reference diffuse reflectance              
R_d_AC_S = (IFT/IFT_h)*R_AC_ref_s
R_d_DC_S = (IFT2/IFT2_l)*R_DC_ref_s


#plotting the diffuse reflectance AC and DC values
plt.subplot(221), plt.imshow(R_d_AC_S), plt.colorbar(), plt.title('AC'), plt.clim(0,0.4), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC_S), plt.colorbar(), plt.title('DC'), plt.xticks([]), plt.yticks([])




#GETTING OPTICAL PROEPRTIES
xi = [] #array of diffuse reflectance DC and AC values from image taken
for i in range(R_d_DC_S.shape[0]): 
    for j in range(R_d_DC_S.shape[1]):
        freq = [R_d_DC_S[i][j], R_d_AC_S[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001) #setting absorption coefficient range
mu_sp = np.arange(0.3, 3, 0.01) #setting reduced scattering coefficient range
n = 1.4 #refractive index of tissue 

#Diffuse reflectance AC value calculated using the Diffusion Approximation
Reflectance_AC = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        
#Diffuse reflectance DC value calculated using the Diffusion Approximation 
#corresponding optical properties for diffuse reflectance values
Reflectance_DC = []
op_mua = []
op_sp = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


#putting DC and AC diffuse reflectance values calculated using the Diffusion Approximation into an array
points = []
for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

#putting optical properties into two seperate arrays
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array


#using scipy.interpolate.griddata to perform cubic interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_abs3 = griddata(points_array, values1, xi, method='cubic') #mua
coeff_sct3 = griddata(points_array, values2, xi, method='cubic') #musp

abs_plot3 = np.reshape(coeff_abs3, (R_d_DC_S.shape[0], R_d_DC_S.shape[1]))
sct_plot3 = np.reshape(coeff_sct3, (R_d_DC_S.shape[0], R_d_DC_S.shape[1]))


#plotting optical property maps
plt.subplot(223), plt.imshow(abs_plot3), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.15)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3), plt.title('Reduced Scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  336
num1:  336
LBP:  308
RBP:  364
Difference:  28


In [5]:
#3
#Hilbert Transform Method
#SSOP to get optical properties
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import scipy
from scipy.signal import hilbert
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('im_1_2_1.tif',0)).astype(np.double) #single image

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

#2D Fourier Transform
f1 = np.fft.fft2(im1) 
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1)) #just used for plotting
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2)) #just used fro plotting


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)



#FOR IMAGE
#HIGH PASS FILTER
#only the AC image requires an additional Hilbert transform - the DC is just inverse Fourier Transform
mask1 = FT1 
mask1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking the centre low frequency
IFT_AC = np.fft.ifft2(np.fft.ifftshift(mask1)) #inverse FFT

Real = np.real(IFT_AC) #selecting real part of inverse Fourier transform
Imag = np.imag(IFT_AC) #selecting imaginary part of inverse Fourier transform

Hilb_r = scipy.signal.hilbert(Real, axis=1) #performing hilbert transform on real part of IFT
Hilb_i = scipy.signal.hilbert(Imag, axis=1) #performing hilbert transform on imaginary part of IFT

Signal = (Hilb_r + 1j*Hilb_i)/2 #adding the real and imaginary parts together and dividing by two

AC_sig = np.abs(Signal) #AC modulation amplitude



#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0 #masking out LHS high frequency
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0 #masking out RHS high frequency

IFT_DC = np.fft.ifft2(np.fft.ifftshift(maskl2)) #performing inverse Fourier Transform
DC_sig = np.abs(IFT_DC) #DC modulation amplitude



#we need to figure out diffuse reflectance of phantom given the optical properties
ref_im = (cv2.imread('im04.tif',0)).astype(np.double) #phantom image
f_ref = np.fft.fft2(ref_im) #2D Fourier Transform
#for high pass filter
FTh = np.fft.fftshift(f_ref)
#for low pass filter
FTl = np.fft.fftshift(f_ref)



#FOR IMAGE
#HIGH PASS FILTER
mask_AC = FTh 
mask_AC[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking the centre low frequency 
IFT_h = np.fft.ifft2(np.fft.ifftshift(mask_AC)) #inverse FFT

Real_ref = np.real(IFT_h) #taking real part of inverse Fourier Transform
Imag_ref = np.imag(IFT_h) #taking imaginary part of inverse Fourier Transform

Hilb_r_ref = scipy.signal.hilbert(Real_ref, axis=1) #performing Hilbert transform on real part of IFT
Hilb_i_ref = scipy.signal.hilbert(Imag_ref, axis=1) #performing Hilbert transform on imaginary part of IFT

Signal_ref = (Hilb_r_ref + 1j*Hilb_i_ref)/2 #adding the real and imaginary parts together and dividing by two

AC_sig_ref = np.abs(Signal_ref) #AC reference modulation amplitude


#LOW PASS FILTER
mask_DC = FTl
mask_DC[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0
mask_DC[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0
IFT2_DC = np.fft.ifft2(np.fft.ifftshift(mask_DC))
DC_sig_ref = np.abs(IFT2_DC) #DC reference modulation amplitude


#to get ref diffuse reflectance
mu_a = 0.0059 #/mm
mu_sp = 0.9748 #/mm
n = 1.4 #refractive index of tissue 

#AC reference diffuse reflectance
f2 = 0.2 #mm^-1
R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
A = (1-R_eff)/(2*(1+R_eff))
mu_tr = mu_a + mu_sp
ap = mu_sp/mu_tr
mu_effp = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)
R_AC_ref_sh = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))

#DC reference diffuse reflectance   
f1 = 0 #mm^-1
mu_effp1 = mu_tr*(3*(mu_a/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)
R_DC_ref_sh = (3*A*ap)/(((mu_effp1/mu_tr)+1)*((mu_effp1/mu_tr)+3*A))


#Diffuse reflectance = (modulation amplitude/reference modulation amplitude)*reference diffuse reflectance     
R_d_AC_Sh = (AC_sig/AC_sig_ref)*R_AC_ref_sh
R_d_DC_Sh = (DC_sig/DC_sig_ref)*R_DC_ref_sh

#plotting diffuse reflectance maps
plt.subplot(221), plt.imshow(R_d_AC_Sh), plt.title('AC'), plt.colorbar(), plt.clim(0,0.4), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(R_d_DC_Sh), plt.title('DC'), plt.colorbar(), plt.xticks([]), plt.yticks([])


#putting diffuse reflectance values from image into an array
xi = []
for i in range(R_d_AC_Sh.shape[0]): 
    for j in range(R_d_AC_Sh.shape[1]):
        freq = [R_d_DC_Sh[i][j], R_d_AC_Sh[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001) #setting absorption coefficient range
mu_sp = np.arange(0.3, 3, 0.01) #setting reduced scattering coefficient range
n = 1.4 #refractive index of tissue 

#AC diffuse reflectance from Diffusion Approximation
Reflectance_AC = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)
        

#DC diffuse reflectance from the Diffusion Approximation
#corresponding optical properties from Diffusion Approximation
Reflectance_DC = []
op_mua = []
op_sp = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)


#putting DC and AC diffuse reflectance values from the Diffusion Approximation into an array
points = []
for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_arrayh = np.array(points)

#putting optical property values into an array
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array

#using scipy.interpolate.griddata to perform cubic interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_absh = griddata(points_arrayh, values1, xi, method='cubic') #mua
coeff_scth = griddata(points_arrayh, values2, xi, method='cubic') #musp

abs_ploth = np.reshape(coeff_absh, (R_d_AC_Sh.shape[0], R_d_AC_Sh.shape[1]))
sct_ploth = np.reshape(coeff_scth, (R_d_AC_Sh.shape[0], R_d_AC_Sh.shape[1]))


#plotting optical property maps
plt.subplot(223), plt.imshow(abs_plot3), plt.title('Absorption $\mu_a$'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.15)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_plot3), plt.title('Reduced Scattering $\mu_s$`'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  336
num1:  336
LBP:  308
RBP:  364
Difference:  28


In [7]:
#4
#performing ssop using hilbert method on image I took in lab with bench top system
#SSOP to get optical properties
#These results are NOT calibrated
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import scipy
from scipy.signal import hilbert
from scipy.interpolate import griddata

%matplotlib qt

im1 = (cv2.imread('Image1.jpg',0)).astype(np.double) #image I took with bench top system in lab

height = int(im1.shape[0]) 
width = int(im1.shape[1])
row = int(im1.shape[0]/2)
col = int(im1.shape[1]/2) 

#2D Fourier transform
f1 = np.fft.fft2(im1) 
#for high pass filter
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1)) #just for plotting
#for low pass filter
FT2 = np.fft.fftshift(f1)
FT2p = 20*np.log(np.abs(FT2)) #just for plotting


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+4:width]))[0][0] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)


#FOR IMAGE
#HIGH PASS FILTER
mask1 = FT1 
mask1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking out centre low frequency

IFT = np.fft.ifft2(np.fft.ifftshift(mask1)) #inverse FFT

Real = np.real(IFT)
Imag = np.imag(IFT)

Hilb_r = scipy.signal.hilbert(Real, axis=1)
Hilb_i = scipy.signal.hilbert(Imag, axis=1)

Signal = (Hilb_r + 1j*Hilb_i)/2

#INSTEAD of calibration with phantoms
AC_sig = np.abs(Signal)/(np.max(np.abs(Signal))/0.3) #this step is assuming a perfect system

#LOW PASS FILTER
maskl2 = FT2
maskl2[0:height, lbp1-int(diff1/2):cbp1-int(diff1/2)] = 0 #masking out LHS high frequency
maskl2[0:height, cbp1+int(diff1/2):rbp1+int(diff1/2)] = 0 #masking out RHS high frequency

IFT2 = np.abs(np.fft.ifft2(np.fft.ifftshift(maskl2))) 
DC_plot = IFT2/np.max(IFT2) #assuming a perfect system, instead of phantom calibration step


#plotting AC and DC diffuse reflectance maps
plt.subplot(221), plt.imshow(AC_sig), plt.title('AC'), plt.colorbar(), plt.xticks([]), plt.yticks([])
plt.subplot(222), plt.imshow(DC_plot), plt.title('DC'), plt.colorbar(), plt.xticks([]), plt.yticks([])

#putting DC and AC diffuse reflectance values in an array
xi = []
for i in range(DC_plot.shape[0]): #does row first and loops through all columns, moves onto next row
    for j in range(DC_plot.shape[1]):
        freq = [DC_plot[i][j], AC_sig[i][j]]
        xi.append(freq)
    

#Getting array of reflectance values and corresponding optical properties
mu_a = np.arange(0, 0.3, 0.001) #setting absorption coefficient range
mu_sp = np.arange(0.3, 3, 0.01) #setting reduced scattering coefficient range
n = 1.4 #refractive index of tissue 

#Diffuse reflectance AC from diffusion Approximation
Reflectance_AC = []
for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f2 = 0.2 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f2)**2)/mu_tr**2)**(1/2)

        R_d1 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_AC.append(R_d1)

#Diffuse reflectance DC from diffusion approximation
#optical properties corresponding to AC and DC diffuse reflectance values
Reflectance_DC = []
op_mua = []
op_sp = []

for i in range(len(mu_a)):
    for j in range(len(mu_sp)):
        f1 = 0 #mm^-1
        R_eff = 0.0636*n + 0.668 + 0.710/n - 1.44/(n**2)
        A = (1-R_eff)/(2*(1+R_eff))
        mu_tr = mu_a[i] + mu_sp[j]
        ap = mu_sp[j]/mu_tr
        mu_effp = mu_tr*(3*(mu_a[i]/mu_tr) + ((2*np.pi*f1)**2)/mu_tr**2)**(1/2)

        R_d2 = (3*A*ap)/(((mu_effp/mu_tr)+1)*((mu_effp/mu_tr)+3*A))
        Reflectance_DC.append(R_d2)
                
        op2 = mu_a[i]
        op_mua.append(op2)
        
        op3 = mu_sp[j]
        op_sp.append(op3)

        
        
#putting DC and AC diffuse reflectance values from Diffusion approximation into an array
points = []
for k in range(len(mu_a)*len(mu_sp)): 
    freq = [Reflectance_DC[k], Reflectance_AC[k]]
    points.append(freq)

points_array = np.array(points)

#optical properties array
op_mua_array = np.array(op_mua) #values1
op_sp_array = np.array(op_sp) #values2

values1 = op_mua_array
values2 = op_sp_array

#using scipy.interpolate.griddata to perform cubic interpolation of diffuse reflectance values to match 
#the generated diffuse reflectance values from image to calculated optical properties
coeff_absi = griddata(points_array, values1, xi, method='cubic') #mua
coeff_scti = griddata(points_array, values2, xi, method='cubic') #musp

abs_ploti = np.reshape(coeff_absi, (DC_plot.shape[0], DC_plot.shape[1]))
sct_ploti = np.reshape(coeff_scti, (DC_plot.shape[0], DC_plot.shape[1]))


#plotting optical property maps
plt.subplot(223), plt.imshow(abs_ploti), plt.title('Absorption'), plt.xticks([]),  plt.yticks([]), plt.clim(0, 0.005)
plt.colorbar()

plt.subplot(224), plt.imshow(sct_ploti), plt.title('Reduced Scattering'), plt.xticks([]),  plt.yticks([])
plt.colorbar()


CBP:  240
num1:  240
LBP:  211
RBP:  268
Difference:  28


In [9]:
#5
#Fourier Transform Profilometry (FTP)
#getting phase
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
from unwrap import unwrap
import scipy
from mpl_toolkits import mplot3d
import matplotlib
from scipy.signal import hilbert

%matplotlib qt

#images taken in Blender
im1_block = (cv2.imread('Blender_sphere.png',0)).astype(np.double) #sine pattern with object 
im2_sine = (cv2.imread('Blender_sine.png',0)).astype(np.double) #w/o object to be imaged

#images taken in lab
#im1_block = (cv2.imread('PiCode_3DFinger_CROP.jpg',0)).astype(np.double) #sine pattern with object 
#im2_sine = (cv2.imread('PiCode_3DSine_CROP.jpg',0)).astype(np.double) #w/o object to be imaged

height = int(im1_block.shape[0]) 
width = int(im1_block.shape[1])

#2D Fourier Transform for image containing object
f1 = np.fft.fft2(im1_block)
FT1 = np.fft.fftshift(f1)
FT1p = 20*np.log(np.abs(FT1))


FT_pd1 = pd.DataFrame(FT1p) #whole image
cbp1 = (np.where(FT_pd1 == np.max(FT1p))[1][0])
print("CBP: ", cbp1)
FT_maxrow1 = FT1p[np.where(FT_pd1 == np.max(FT1p))[1][0]]
num1 = np.where(FT_pd1 == np.max(FT1p))[1][0]
print("num1: ", num1)
lbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[0:num1-5]))[0][0] #left bright point
print("LBP: ", lbp1)
rbp1 = np.where(FT_maxrow1 == np.max(FT_maxrow1[num1+5:width]))[0][1] #right bright point
print("RBP: ", rbp1)
diff1 = int(rbp1 - cbp1)
print("Difference: ", diff1)


#high pass filter
FT1[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking out low frequency
IFT = np.fft.ifft2(np.fft.ifftshift(FT1))

Real1 = np.real(IFT)
Imag1 = np.imag(IFT)

Hilb_r1 = scipy.signal.hilbert(Real1, axis=1)
Hilb_i1 = scipy.signal.hilbert(Imag1, axis=1)

Signal1 = (Hilb_r1 + 1j*Hilb_i1)/2 #image containing object


##2D Fourier Transform for image of just sine pattern
f2 = np.fft.fft2(im2_sine)
FT2 = np.fft.fftshift(f2) 
FT2p = 20*np.log(np.abs(FT2))

#high pass filter
FT2[0:height, cbp1-int(diff1/2):cbp1+int(diff1/2)] = 0 #masking out low frequency
IFT2 = np.fft.ifft2(np.fft.ifftshift(FT2))

Real2 = np.real(IFT2)
Imag2 = np.imag(IFT2)

Hilb_r2 = scipy.signal.hilbert(Real2, axis=1)
Hilb_i2 = scipy.signal.hilbert(Imag2, axis=1)

Signal2 = (Hilb_r2 + 1j*Hilb_i2)/2 


#obtaining phase from AC modulation amplitudes 
fi = np.imag(np.log(Signal1*np.conj(Signal2)))
#plt.subplot(131), plt.imshow(fi), plt.colorbar(), plt.title('Phase')

#unwrapping phase
unwrap = unwrap(fi, wrap_around_axis_0=False, wrap_around_axis_1=False, wrap_around_axis_2=False)
#plt.subplot(132), plt.imshow(np.abs(unwrap)), plt.colorbar(), plt.title('Abs(Unwrapped Phase)'), plt.clim(0,3)


phase_h = []

#FOR BLENDER SETUP3
#calculating height
for j in range(im1_block.shape[0]):
    for i in range(im1_block.shape[1]):
        l_0 = 8000 #distance from camera to reference frame[mm]
        ph = np.abs(unwrap[j][i]) #calculated phase
        f_0 = 0.01 #/mm - fundamental frequency of observed grating image
        d = 560 #distance between projector and camera
        phase_height = (l_0*ph)/(ph-2*np.pi*f_0*d)
        phase_h.append(phase_height)
        
        
height = np.reshape(phase_h, (im1_block.shape[0],im1_block.shape[1])) #reshaping height to image format
absheight = np.abs(height)

#removing outliers from image
for i in range(im1_block.shape[0]):
    for j in range(im1_block.shape[1]):
        if absheight[i][j] > 820:
            absheight[i][j] = 0
            

            
'''
#FOR IMAGES TAKEN WITH BENCH TOP SYSTEM
#calculating height
for j in range(im1_block.shape[0]):
    for i in range(im1_block.shape[1]):
        l_0 = 200 #distance from camera to reference frame[mm]
        ph = np.abs(unwrap[j][i]) #calculated phase
        f_0 = 0.2 #/mm - fundamental frequency of observed grating image
        d = 30 #distance between projector and camera
        phase_height = (l_0*ph)/(ph-2*np.pi*f_0*d)
        phase_h.append(phase_height)
        
height = np.reshape(phase_h, (im1_block.shape[0],im1_block.shape[1])) #reshaping height to image format
absheight = np.abs(height)

#remvoing outliers from image
for i in range(im1_block.shape[0]):
    for j in range(im1_block.shape[1]):
        if absheight[i][j] > 45:
            absheight[i][j] = 0
'''        
        
#plt.subplot(133), plt.imshow(absheight), plt.colorbar(), plt.xticks([]), plt.yticks([]), plt.clim(0,5)


#3D plot of height
fig = plt.figure()
ax = plt.axes(projection='3d')
x = np.linspace(0, im1_block.shape[1], im1_block.shape[1])
y = np.linspace(0, im1_block.shape[0], im1_block.shape[0])
X, Y = np.meshgrid(x, y)
ax.view_init(40, 270) #(q,r) q degrees above x-y plane and rotated r degrees counter clockwise around z #40,270
ax.plot_surface(X, Y, absheight, rstride=1, cstride=1, edgecolor='none', cmap = 'viridis'),ax.set_xlabel('x'), ax.set_ylabel('y'), ax.set_zlabel('z')


CBP:  500
num1:  500
LBP:  481
RBP:  519
Difference:  19


(<mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x1c3fb9767c8>,
 Text(0.5, 0, 'x'),
 Text(0.5, 0, 'y'),
 Text(0.5, 0, 'z'))